In [1]:
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd

os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP')


In [2]:

# NHẬP - XUẤT - TỒN & EXCHANGE & GIÁ VỐN & MUA HÀNG & BÁN HÀNG

data_nhapxuatton=pd.read_excel("../VACP/Export BRAVO/Tổng hợp Nhập - Xuất - Tồn YTD.xls",
                        usecols="A:B,G:O",skipfooter=1,header=5).rename(columns={
                                "Unnamed: 0":"ID",
                                "Unnamed: 1":"Tên vật tư",
                                "Số lượng":"Tổng đầu kỳ", 
                                "Giá trị":"Giá trị đầu kỳ",
                                "Số lượng.1":"Tổng nhập", 
                                "Giá trị.1":"Giá trị nhập",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất",
                                "Số lượng.3":"Tổng tồn", 
                                "Giá trị.3":"Giá trị tồn" })


data_muahang=pd.read_excel("../VACP/Export BRAVO/Sổ chi tiết mua hàng (2).xls",
                                usecols="A,C,F,I:J",skipfooter=1,header=5).rename(columns={
                                "Unnamed: 0":"Ngày nhập",
                                "Unnamed: 2":"Nhà cung cấp",
                                "Unnamed: 5":"ID",
                                "Số lượng":"Tổng nhập", 
                                "Đơn giá":"Giá nhập"
                                })

data_banhang=pd.read_excel("../VACP/Export BRAVO/Sổ chi tiết bán hàng.xls",
                        usecols="A,C:D,F,H,K:M",skipfooter=1,header=6).rename(columns={
                                "Unnamed: 0":"Ngày bán",
                                "Unnamed: 2":"ID Khách hàng",
                                "Unnamed: 3":"Hãng",
                                "Unnamed: 5":"ID",
                                "Unnamed: 7":"Tên hàng",
                                "Số lượng":"Tổng bán", 
                                "Đơn giá gốc":"Đơn giá bán",
                                "Giá bán":"Giá bán giảm"
                                })

data_danhmuchanghoa=pd.read_excel("../VACP/Python NCC Min/Danh mục vật tư hàng hóa .xls",usecols="A,C:D,M").rename(columns={
                                "Unnamed: 0":"ID Nhóm hàng",
                                "Unnamed: 2":"ID",
                                "Unnamed: 3":"Tên vật tư",
                                "Unnamed: 12":"Hãng SX"
                                }).dropna(subset=['ID','ID Nhóm hàng']).reset_index(drop=True)

C:\Users\TAM\AppData\Local\Temp\ipykernel_14056\4109355123.py:3: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  data_nhapxuatton=pd.read_excel("../VACP/Export BRAVO/Tổng hợp Nhập - Xuất - Tồn YTD.xls",


In [3]:
data_giaban=pd.read_excel("../VACP/Export BRAVO/Báo cáo giá bán, tồn kho.xls",usecols="B,C,F",skipfooter=1,header=4).rename(
    columns={
        "Tên sub nhóm":"ID Nhóm hàng",
        "Mã số":"ID"
    }
).dropna(subset=['ID','ID Nhóm hàng']).reset_index(drop=True)
 

In [4]:
# Giá vốn
TransCost=0.05
data_nhapxuatton['Giá vốn'] =(1+TransCost)*np.where(data_nhapxuatton["Tổng xuất"]>0,
                                        data_nhapxuatton["Giá trị xuất"]/data_nhapxuatton["Tổng xuất"],
                                            np.where(data_nhapxuatton["Tổng tồn"]>0,
                                                data_nhapxuatton["Giá trị tồn"]/data_nhapxuatton["Tổng tồn"],0

                                            )
)

#Mua hàng giá nhập nhỏ nhất
min_mua_hang=data_muahang.groupby(["ID"])["Giá nhập"].min()
data_muahang_min=data_muahang.merge(min_mua_hang,on="ID",how="left",suffixes=('', '_min'))
data_muahang_min=data_muahang_min[data_muahang_min["Giá nhập"]==data_muahang_min["Giá nhập_min"]].drop('Giá nhập_min',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)
data_muahang_min=data_muahang_min.rename(columns={"Tổng nhập":"Tổng nhập MIN","Giá nhập":"Giá nhập MIN","Ngày nhập":"Ngày nhập MIN","Nhà cung cấp":"Nhà cung cấp MIN"})

#Mua hàng giá nhập gần nhất
max_date_muahang=data_muahang.groupby(["ID"])["Ngày nhập"].max()
data_muahang_latest=data_muahang.merge(max_date_muahang,on="ID",how="left",suffixes=('', '_max'))
data_muahang_latest=data_muahang_latest[data_muahang_latest["Ngày nhập"]==data_muahang_latest["Ngày nhập_max"]].drop('Ngày nhập_max',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)
data_muahang_latest=data_muahang_latest.rename(columns={"Tổng nhập":"Tổng nhập gần đây","Giá nhập":"Giá nhập gần đây","Ngày nhập":"Ngày nhập gần đây","Nhà cung cấp":"Nhà cung cấp gần đây"})


#Bán hàng gần đây
max_date_banhang=data_banhang.groupby(["ID"])["Ngày bán"].max()
data_banhang_latest=data_banhang.merge(max_date_banhang,on="ID",how="left",suffixes=('', '_max'))
data_banhang_latest=data_banhang_latest[data_banhang_latest["Ngày bán"]==data_banhang_latest["Ngày bán_max"]].drop('Ngày bán_max',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)



In [5]:
# # Gộp dữ liệu Nhập Xuất Tồn + MUA Min Gân đây+ BÁN
data_nhap_mua_ban=data_nhapxuatton.merge(data_giaban[["ID","Giá bán"]],on="ID",how='left').merge(
            data_danhmuchanghoa[["ID","ID Nhóm hàng","Hãng SX"]],on="ID",how='left').merge(
                            data_muahang_min,on="ID",how='left').merge(
                                    data_muahang_latest,on="ID",how='left').merge(
                                            data_banhang_latest,on="ID",how='left')
                                            


 

In [6]:

# Thuật toán ABC

# https://www.kaggle.com/code/dongxy/inventory-management-abc-analysis-python
# Phân tích ABC Analysis

# create 3 grades A,B,C based on the running percentage (A-60%,B-25%,C-15%)
def ABC_segmentation(RunPerc):
    if RunPerc > 0 and RunPerc < 0.6:
        return 'A'
    elif RunPerc >=0.6 and RunPerc < 0.85:
        return 'B'
    elif RunPerc >=0.85:
        return 'C'
    
    
data_nhap_mua_ban=data_nhap_mua_ban.sort_values("Giá trị xuất",ascending=False)

#--------------------------- Tỷ trọng giá trị hàng xuất -----------------------------------------
# create the column of the running CumCost of the cumulative cost per SKU
data_nhap_mua_ban["Cum xuất"]=data_nhap_mua_ban["Giá trị xuất"].cumsum()

# create the column of the total sum
data_nhap_mua_ban["Total xuất"] = data_nhap_mua_ban["Giá trị xuất"].sum()

# create the column of the running percentage 
data_nhap_mua_ban["Tỷ trọng xuất"] = data_nhap_mua_ban["Cum xuất"]/data_nhap_mua_ban["Total xuất"]


# create the column of the class
data_nhap_mua_ban['Class'] = data_nhap_mua_ban['Tỷ trọng xuất'].apply(ABC_segmentation)


In [7]:
data_report=data_nhap_mua_ban[[
                    "ID","Tên vật tư",
                    "ID Nhóm hàng","Hãng SX",
                    "Giá vốn","Giá bán","Giá bán giảm",
                    "Ngày nhập MIN","Nhà cung cấp MIN","Giá nhập MIN","Tổng nhập MIN"
                    "Ngày nhập gần đây","Nhà cung cấp gần đây","Giá nhập gần đây","Tổng nhập gần đây",
                    "Ngày bán","Tổng bán",
                    "Tổng đầu kỳ","Tổng nhập","Tổng xuất", "Tổng tồn",
                    "Giá trị đầu kỳ","Giá trị nhập","Giá trị xuất","Giá trị tồn",
                    "Class"
                    ]]

data_report=data_report.reset_index(drop=True)

In [8]:
# data_2022=pd.read_excel(
#     "../VACP/Gửi File cho Chị Phương/Hàng xả 1022.xlsx",
#     sheet_name="Năm 2022"
# )

# data_2021=pd.read_excel(
#     "../VACP/Gửi File cho Chị Phương/Hàng xả 1022.xlsx",
#     sheet_name="Năm 2021"
# )

# data_2020=pd.read_excel(
#     "../VACP/Gửi File cho Chị Phương/Hàng xả 1022.xlsx",
#     sheet_name="Năm 2020"
# )

# data_output=pd.concat([data_2020,data_2021,data_2022]).reset_index(drop=True)

In [9]:
# EXPORT FILE DATA
with pd.ExcelWriter('../VACP/Input & Export Campaign/Report_Nhap_Mua_Ban.xlsx') as writer:
    data_report.to_excel(writer, sheet_name='Nhập Mua Bán')
    data_nhap_mua_ban.to_excel(writer,sheet_name='Data Full')
    # data_output.to_excel(writer,sheet_name="output")

